<a href="https://colab.research.google.com/github/jungcheolkwon/gcolab/blob/main/f5sdk_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install f5-sdk-python f5-sdk ipstack pandas awscli

**Add Colab IP address to AWS Security-Group at each regions**
- need to take AWS STS keys before run this scripts

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount("/content/gdrive")
!cp "/content/gdrive/MyDrive/Colab/aws_acl.sh" '/content'
!chmod 755 aws_acl.sh
!./aws_acl.sh

**Lession1. Get BIGIP basic information**

In [ ]:
#Lession1. how to bigip basic information
#2.Dec.2020

from f5.bigip import ManagementRoot
from getpass import getpass
from ipstack import GeoLookup
import pandas as pd
from google.colab import drive

with open('/content/gdrive/MyDrive/Colab/credential.txt', "r") as f:
  cred = f.read().splitlines() 
  user = cred[0];   pwd = cred[1]

# user = input('Enter username:')
# pwd = getpass('Enter admin password')

geo_lookup = GeoLookup("0cbeddca8c7f050ead7d6eff52a73bb4")

data = []
with open('/content/gdrive/MyDrive/Colab/hosts.txt', "r") as f:
  hosts = f.read().splitlines()
  for ip in hosts:
    mgmt = ManagementRoot(ip, user, pwd, port=8443)

    location = geo_lookup.get_location(ip)
    address = location['country_code']
    version = mgmt.tmos_version
    data.append([ip, address, version])
  df = pd.DataFrame(data, columns=["IP", "Location", "Version"])
  print(df)
#remove comment(#) the below lines to save csv file
df.to_csv('/content/gdrive/MyDrive/Colab/device_info.csv')


              IP Location   Version
0    3.121.15.40       DE    15.1.1
1  13.251.170.12       SG    14.1.3
2  34.205.120.66       US  15.0.1.1


**Lession2. Create VS, Pool and Members**

In [ ]:
##Lession2; how to create VS, Pools, Members
##2.Dec.2020

from f5.bigip import ManagementRoot
from getpass import getpass
from ipstack import GeoLookup 
import pandas as pd  

def MyFunction(location,pip,vip):
    my_node = location + "-NODE"
    ltm.nodes.node.create(partiion='Common', name=my_node, address=pip, monitors='http')

    my_pool = location + "-POOL"
    params = {
        'name': my_pool,
        'partition': 'Common',
        'monitor': 'http'
    }
    ltm.pools.pool.create(**params)

    member_port = '80'
    pool = ltm.pools.pool.load(name=my_pool)
    pool.members_s.members.create(partition='Common', name=my_node + ":" + member_port)

    vs_name = location + "-VS"
    params = {
        'name': vs_name,
        'partition': 'Common',
        'source': '0.0.0.0/0',
        'destination': '{}:{}'.format(vip, str(80)),
        'mask': '255.255.255.255',
        'description': 'Created by F5 SDK',
        'protocol': 'tcp',
        'pool': my_pool,
        'profiles': [
          {'name': 'http'},
          {'name': 'tcp'}           
        ],
        'sourceAddressTranslation': {'type': 'automap'},
        'vlansEnabled': True,
        'vlans': ['/Common/internal']
    }
    ltm.virtuals.virtual.create(**params)

    print ('Created objects at"', location, sep=' ') 
    print ("   ")
    return

with open('/content/gdrive/MyDrive/Colab/credential.txt', "r") as f:
  cred = f.read().splitlines() 
  user = cred[0];   pwd = cred[1]

#user = input('Enter username:')
#pwd = getpass('Enter admin password')

with open('/content/gdrive/MyDrive/Colab/hosts.txt', "r") as f:
  hosts = f.read().splitlines()
  for ip in hosts:
    mgmt = ManagementRoot(ip, user, pwd, port=8443)
    ltm = mgmt.tm.ltm

    geo_lookup = GeoLookup("0cbeddca8c7f050ead7d6eff52a73bb4")
    geoinfo = geo_lookup.get_location(ip)
    location = geoinfo['country_code']
    #version = mgmt.tmos_version

    if location == 'SG':
    #if ( location == 'SG' and version == '14.1.3' ):
    #if ( location == 'SG' and version == '15.1.1' ):
      pip='172.31.17.204'
      vip='0.0.0.0'
      MyFunction(location,pip,vip)

    if location == 'DE':
    #if ( location == 'DE' and version == '15.1.1' ):
      pip='172.31.17.180'
      vip='0.0.0.0'
      MyFunction(location,pip,vip)

    if location == 'US':
    #if ( location == 'US' and version =='15.0.1.1' ):
    #if ( location == 'US' and version =='14.1.3' ):
      pip='10.0.0.237' 
      vip='0.0.0.0'
      MyFunction(location,pip,vip)

print ('Completed')

Created objects at" DE
   
Created objects at" SG
   
Created objects at" US
   
Completed


***Lession3. Get VS, Pools and Members ***

In [ ]:
##Lession3; how to Get VS, Pools, Members
##2.Dec.2020

from f5.bigip import ManagementRoot
from getpass import getpass
import pandas as pd 

with open('/content/gdrive/MyDrive/Colab/credential.txt', "r") as f:
  cred = f.read().splitlines() 
  user = cred[0];   pwd = cred[1]

#user = input('Enter username:')
#pwd = getpass('Enter admin password')

data = []
with open('/content/gdrive/MyDrive/Colab/hosts.txt', "r") as f:
  hosts = f.read().splitlines()
  for ip in hosts:
    mgmt = ManagementRoot(ip, user, pwd, port=8443)
    pools = mgmt.tm.ltm.pools.get_collection()
    virtuals = mgmt.tm.ltm.virtuals.get_collection()

    for pool in pools:
      POOLS = pool.name
      for member in pool.members_s.get_collection():
        MEMBERS = member.name
        for virtual in virtuals:
          VS = virtual.name
        data.append([VS, POOLS, MEMBERS])
df = pd.DataFrame(data, columns=["VS", "POOLS", "MEMBERS"])
print(df)
#remove comment(#) the below lines to save csv file
df.to_csv('/content/gdrive/MyDrive/Colab/device_pools.csv')

      VS    POOLS     MEMBERS
0  DE-VS  DE-POOL  DE-NODE:80
1  SG-VS  SG-POOL  SG-NODE:80
2  US-VS  US-POOL  US-NODE:80


**Lession4. Delete Objects**

In [ ]:
#Lession4. Delete objects
##2.Dec.2020

from f5.bigip import ManagementRoot
from getpass import getpass
from ipstack import GeoLookup 
import pandas as pd 

with open('/content/gdrive/MyDrive/Colab/credential.txt', "r") as f:
  cred = f.read().splitlines() 
  user = cred[0];   pwd = cred[1]

#user = input('Enter username:')
#pwd = getpass('Enter admin password')

with open('/content/gdrive/MyDrive/Colab/hosts.txt', "r") as f:
  hosts = f.read().splitlines()
  for ip in hosts:
    mgmt = ManagementRoot(ip, user, pwd, port=8443)
    ltm = mgmt.tm.ltm

    geo_lookup = GeoLookup("0cbeddca8c7f050ead7d6eff52a73bb4")
    geoinfo = geo_lookup.get_location(ip)
    location = geoinfo['country_code']

    vs_name = location + "-VS"
    if ltm.virtuals.virtual.exists(partition='Common', name=vs_name):
        virtual = ltm.virtuals.virtual.load(
            partition='Common', name=vs_name)
        virtual.delete()
    else:
        print("Virtual server does not exist")
        break

    my_pool = location + "-POOL"
    ltm.pools.pool.load(partition='Common', name=my_pool).delete()

    my_node = location + "-NODE"
    ltm.nodes.node.load(partition='Common', name=my_node).delete()

    print ("Deleted objects from" + "  " + location)
    print ("   ")
print ("completed!!")

**Lession5. AS3 with SDK to create/delete Objects**

In [ ]:
#Lession5. AS3 with SDK to create and delete objects
##4.Dec.2020

from f5sdk.bigip import ManagementClient
from f5sdk.bigip.extension import AS3Client, DOClient, TSClient, CFClient
from google.colab import drive
from getpass import getpass

#drive.mount("/content/gdrive", force_remount=True)

with open('/content/gdrive/MyDrive/Colab/credential.txt', "r") as f:
  cred = f.read().splitlines() 
  user = cred[0];   pwd = cred[1]

#user = input('Enter username: ')
#pwd = getpass('Enter admin password')
action = input('Do you want to add or del? Type add or del')

def Action(action,location):
    if action == 'add':
      if location == 'SG':
        extension_client.service.create(config_file='/content/gdrive/MyDrive/Colab/as3-sg2.json')
        print ("AS3 Template is just loaded at"+ " "+location)

      if location == 'DE':
        extension_client.service.create(config_file='/content/gdrive/MyDrive/Colab/as3-de.json')
        print ("AS3 Template is just loaded at"+ " "+location)

      if location == 'US':
        extension_client.service.create(config_file='/content/gdrive/MyDrive/Colab/as3-us.json')
        print ("AS3 Template is just loaded at"+ " "+location)
      
    if action == 'del':
      if location == 'SG':
        extension_client.service.create(config_file='/content/gdrive/MyDrive/Colab/as3_del.json')
        print ("AS3 Configuration is just deleted at"+ " "+location)

      if location == 'DE':
        extension_client.service.create(config_file='/content/gdrive/MyDrive/Colab/as3_del.json')
        print ("AS3 Configuration is just deleted at"+ " "+location)

      if location == 'US':
        extension_client.service.create(config_file='/content/gdrive/MyDrive/Colab/as3_del.json')
        print ("AS3 Configuration is just deleted at"+ " "+location)
    return

with open('/content/gdrive/MyDrive/Colab/hosts.txt', "r") as f:
  hosts = f.read().splitlines()
  for ip in hosts:
    mgmt_client = ManagementClient(ip, user=user, password=pwd)
    extension_client = AS3Client(mgmt_client)

    geo_lookup = GeoLookup("0cbeddca8c7f050ead7d6eff52a73bb4")
    geoinfo = geo_lookup.get_location(ip)
    location = geoinfo['country_code']
    if action == 'add' or action == 'del':
      Action(action,location)

print ("Completed")

Lession6. Take statistics

In [ ]:
#Lession5. Take pools statistics info
##6.Dec.2020

from f5.bigip import ManagementRoot
from getpass import getpass
from ipstack import GeoLookup 
import pandas as pd 

with open('/content/gdrive/MyDrive/Colab/credential.txt', "r") as f:
  cred = f.read().splitlines() 
  user = cred[0];   pwd = cred[1]

#user = input('Enter username: ')
#pwd = getpass('Enter admin password')

data = []
with open('/content/gdrive/MyDrive/Colab/hosts.txt', "r") as f:
  hosts = f.read().splitlines()
  for ip in hosts:
    mgmt = ManagementRoot(ip, user, pwd, port=8443)
    ltm = mgmt.tm.ltm

    geo_lookup = GeoLookup("0cbeddca8c7f050ead7d6eff52a73bb4")
    geoinfo = geo_lookup.get_location(ip)
    location = geoinfo['country_code']
    
    my_pool = location + "-POOL"
    partition = 'Common'

    print ('-----', my_pool, '-----', sep=' ')
    
    pool  = ltm.pools.pool.load(name=my_pool, partition=partition)
    stats = pool.stats.load()

    for item in stats.entries:
      params = stats.entries.get(item).get('nestedStats').get('entries')
      stuff = ['activeMemberCnt','curSessions','serverside.curConns','serverside.maxConns','serverside.totConns','status.enabledState','status.availabilityState']
 
    for item in stuff:
      for key in ('value','description'):
        if key in params[item]:
          print("{}: {}".format(item,params[item][key]))
          #df = pd.DataFrame(data, format(item,params[item][key]))
          #print(df)
#remove comment(#) the below lines to save csv file
#df.to_csv('/content/gdrive/MyDrive/Colab/pools_statistics.csv')
 